# ECS7020P mini-project

**Student Name**:  Shahin Mammadov
**Student ID**:  240315623



In [3]:
pip install mlend==1.0.0.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 15.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.5 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: python-pica

In [4]:
import mlend
from mlend import download_deception_small, deception_small_load

In [5]:
datadir = download_deception_small(save_to='MLEnd', subset={}, verbose=1, overwrite=False)

100%|▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓|100\100|00100.wav
Done!


In [6]:
TrainSet, TestSet, MAPs = deception_small_load(datadir_main=datadir, train_test_split=None, verbose=1, encode_labels=True)

Total 100 found in MLEnd/deception/MLEndDD_stories_small/


In [7]:
pip install librosa


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install numpy


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import librosa
import numpy as np  
import pandas as pd
import random

"""                
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    return np.hstack([np.mean(mfccs, axis=1), np.mean(chroma, axis=1), np.mean(zcr)])
"""

def clean_audio(file_path, target_sr=16000):
    y, sr = librosa.load(file_path, sr=target_sr, mono=True)
    y = librosa.effects.trim(y, top_db=20)[0]  # Remove silence
    y = librosa.util.normalize(y)  # Normalize amplitude
    return y

def augment_audio(y, sr):
    # Random pitch shift
    n_steps = random.uniform(-2, 2)
    y = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
    # Random time stretch
    rate=random.uniform(0.8, 1.2)
    y = librosa.effects.time_stretch(y, rate=rate)
    # Add background noise
    noise = np.random.normal(0, 0.005, len(y))
    y = y + noise
    return librosa.util.normalize(y)

def pad_or_truncate(y, target_length=48000):  # 3 seconds at 16 kHz
    if len(y) > target_length:
        return y[:target_length]
    else:
        return np.pad(y, (0, max(0, target_length - len(y))), mode='constant')

def extract_advanced_features(file_path):
    y = clean_audio(file_path)
    y = augment_audio(y, sr=16000)
    y = pad_or_truncate(y)
    
    # Extract features
    mfccs = librosa.feature.mfcc(y=y, sr=16000, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=16000)
    zcr = librosa.feature.zero_crossing_rate(y)
    rms = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=16000)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=16000)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=16000)
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=16000, roll_percent=0.85)
    tempo, _ = librosa.beat.beat_track(y=y, sr=16000)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=16000, n_mels=128)
    
    # Harmonic and Percussive components
    harmonic, percussive = librosa.effects.hpss(y)
    
    print(f"{len(chroma)} {len(spectral_contrast)}")

    # Combine features
    features = np.hstack([
        np.mean(mfccs, axis=1),
        np.mean(chroma, axis=1),
        np.mean(zcr),
        np.mean(rms),
        np.mean(spectral_centroid),
        np.mean(spectral_bandwidth),
        np.mean(spectral_contrast, axis=1),
        np.mean(spectral_flatness),
        np.mean(spectral_rolloff),
        tempo,
        np.mean(harmonic),
        np.mean(percussive),
        np.mean(mel_spectrogram, axis=1)
    ])
    return features



In [15]:
data = pd.read_csv('MLEnd/deception/MLEndDD_story_attributes_small.csv')

In [16]:
X, y = [], []

for _, row in data.iterrows():
    features = extract_advanced_features(f"MLEnd/deception/MLEndDD_stories_small/{row['filename']}")
    X.append(features)
    y.append(row['Story_type'])

12 7
12 7
12 7
12 7
12 7
12 7
12 7
12 7
12 7
12 7
12 7


KeyboardInterrupt: 

In [10]:
len(features)

196

In [21]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

X = np.array(X)
y = LabelEncoder().fit_transform(y)

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Train model
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.52


In [29]:
pip install lightgbm


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# LightGBM parameters
params = {
    'objective': 'binary',  # For binary classification
    'metric': 'accuracy',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train model
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Predict and evaluate
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_binary))


[LightGBM] [Info] Number of positive: 43, number of negative: 32
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000488 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5111
[LightGBM] [Info] Number of data points in the train set: 75, number of used features: 196
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.573333 -> initscore=0.295464
[LightGBM] [Info] Start training from score 0.295464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [57]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', probability=True, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred))

SVM Accuracy: 0.28


In [58]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Train XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))


XGBoost Accuracy: 0.56


/Users/shahinmammadov/.pyenv/versions/myenv/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:59:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

